In [13]:
import pandas as pd
import requests

In [14]:
symbol_values = ['IBM', 'GOOGL', 'MSFT', 'AAPL', 'AMZN']

# Create request call for for each tickr symbol
for i, val in enumerate(symbol_values):
    url = ''
    url += 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}&outputsize=full&apikey=W3BMUCZT63KDH0ER'.format(val)
    r = requests.get(url)
    data = r.json()
    
    # Create dataframe from json
    df_stockmarket_data = pd.DataFrame(data, index=None)
    
    # Replace index to interger index
    df_stockmarket_data.reset_index(inplace=True)
    
    # Rename index column to date
    df_stockmarket_data['date'] = df_stockmarket_data['index']
    
    # Select value of symbol from meta Data column and add new column for symbol in df
    df_stockmarket_data['symbol'] = df_stockmarket_data.loc[1,'Meta Data']
    
    # repeat previous step for adding new columns for refresh dat, output size and time zone
    df_stockmarket_data['last_refreshed'] = df_stockmarket_data.loc[2,'Meta Data']
    df_stockmarket_data['output_size'] = df_stockmarket_data.loc[3,'Meta Data']
    df_stockmarket_data['time_zone'] = df_stockmarket_data.loc[4,'Meta Data']
    
    # Select and reorder columns in df
    df_stockmarket_data = pd.DataFrame(df_stockmarket_data, columns=['date', 'symbol', 'Time Series (Daily)', 'last_refreshed', 'output_size', 'time_zone'])
    
    # Remove first 5 rows starting from index 0 to 4 as it is metadata
    df_stockmarket_data = df_stockmarket_data[5:]
    
    # Convert time Series Daily column from dictionary to data frame columns
    df_stockmarket_all_columns = pd.concat([df_stockmarket_data.drop('Time Series (Daily)', axis=1), df_stockmarket_data['Time Series (Daily)'].apply(pd.Series)], axis=1)
    
    # Reset index after removing top 5 rows
    df_stockmarket_all_columns = df_stockmarket_all_columns.reset_index().drop('index', axis=1)
    # Rename columns split from dictionary
    df_stockmarket_all_columns.rename(columns = {'1. open':'open', '2. high': 'high', '3. low': 'low', '4. close': 'close', '5. volume':'volume'}  , inplace=True)
    
    # Append data from each ticker symbol to data set created from previous step
    if i == 0:
        
        # Copy rows of current tickr symbol to a copy df
        df_stockmarket_all_columns_copy2 = df_stockmarket_all_columns.copy(deep=True)
        
        # convert to dict
        df_stockmarket_all_columns = pd.DataFrame(df_stockmarket_all_columns)
        
        # Create a new df to append current rows
        df_stockmarket_all_final = pd.concat([df_stockmarket_all_columns,df_stockmarket_all_columns_copy2.loc[:]]).reset_index(drop=True)
        
        # Copy the newly appended dataset to a new dataset which stores the previous state
        df_stockmarket_all_columns_previous = df_stockmarket_all_columns_copy2.copy(deep=True)
    else:
        # convert to dict
        df_stockmarket_all_columns = pd.DataFrame(df_stockmarket_all_columns)
        
        # From second tickr symbol onwards appendf the current rows to data stored from previous step
        df_stockmarket_all_final = pd.concat([df_stockmarket_all_columns,df_stockmarket_all_columns_previous.loc[:]]).reset_index(drop=True)
        
        # Copy the newly appended df into previous_df and repeat the process until we have data for all symbols
        df_stockmarket_all_columns_previous = df_stockmarket_all_final.copy(deep=True)
        
df_stockmarket_all_final.to_json('/Users/meetapandit/DE_Bootcamp/Capstone_1/AlphaAdvantage_stockdata/stockmarket_data.json', orient='records')
df_stockmarket_all_final

             date symbol last_refreshed output_size   time_zone      open  \
0      2023-09-08   AMZN     2023-09-08   Full size  US/Eastern  136.8600   
1      2023-09-07   AMZN     2023-09-08   Full size  US/Eastern  133.9000   
2      2023-09-06   AMZN     2023-09-08   Full size  US/Eastern  136.3200   
3      2023-09-05   AMZN     2023-09-08   Full size  US/Eastern  137.7300   
4      2023-09-01   AMZN     2023-09-08   Full size  US/Eastern  139.4550   
...           ...    ...            ...         ...         ...       ...   
28800  1999-11-05    IBM     2023-09-08   Full size  US/Eastern   92.7500   
28801  1999-11-04    IBM     2023-09-08   Full size  US/Eastern   94.4400   
28802  1999-11-03    IBM     2023-09-08   Full size  US/Eastern   95.8700   
28803  1999-11-02    IBM     2023-09-08   Full size  US/Eastern   96.7500   
28804  1999-11-01    IBM     2023-09-08   Full size  US/Eastern   98.5000   

           high       low     close    volume  
0      138.8519  136.7500  

In [4]:
print(df_stockmarket_all_final.shape[0])

# get unique values from symbol column

print(df_stockmarket_all_final['symbol'].unique())

28805
['AMZN' 'AAPL' 'MSFT' 'GOOGL' 'IBM']


In [6]:
df_agg = df_stockmarket_all_final.groupby(by=['symbol']).count()
df_agg


,date,last_refreshed,output_size,time_zone,open,high,low,close,volume
symbol,,,,,,,,,
AAPL,6002,6002,6002,6002,6002,6002,6002,6002,6002
AMZN,6002,6002,6002,6002,6002,6002,6002,6002,6002
GOOGL,4797,4797,4797,4797,4797,4797,4797,4797,4797
IBM,6002,6002,6002,6002,6002,6002,6002,6002,6002
MSFT,6002,6002,6002,6002,6002,6002,6002,6002,6002


In [7]:
df_stockmarket_all_final.describe()

,date,symbol,last_refreshed,output_size,time_zone,open,high,low,close,volume
count,28805,28805,28805,28805,28805,28805,28805,28805,28805,28805
unique,6002,5,1,1,1,19951,20124,20339,20492,28175
top,2023-09-08,AMZN,2023-09-08,Full size,US/Eastern,92.0000,26.0000,25.5000,27.2500,4178600
freq,5,6002,28805,28805,28805,13,17,17,12,4


In [8]:
df_stockmarket_all_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28805 entries, 0 to 28804
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            28805 non-null  object
 1   symbol          28805 non-null  object
 2   last_refreshed  28805 non-null  object
 3   output_size     28805 non-null  object
 4   time_zone       28805 non-null  object
 5   open            28805 non-null  object
 6   high            28805 non-null  object
 7   low             28805 non-null  object
 8   close           28805 non-null  object
 9   volume          28805 non-null  object
dtypes: object(10)
memory usage: 2.2+ MB
